In [10]:
test_str = str({
    "language": "python",
    "file_name": "calculate_derivative.py",
    "request": "y=e^|x|의 접점의 기울기를 구하기 위한 코드 작성",
    "script": "import sympy as sp\n\nx = sp.symbols('x')\ny = sp.exp(sp.Abs(x))\nderivative = sp.diff(y, x)\n\n# 특정 점에서의 기울기 계산\npoint = 0  # 예를 들어 x=0에서의 기울기\nslope_at_point = derivative.subs(x, point)\nprint(slope_at_point)"
})

In [7]:
test_str = test_str.replace("'", "\"")

In [9]:
test_str

'{"language": "python", "file_name": "calculate_derivative.py", "request": "y=e^|x|의 접점의 기울기를 구하기 위한 코드 작성", "script": "import sympy as sp\\n\\nx = sp.symbols("x")\\ny = sp.exp(sp.Abs(x))\\nderivative = sp.diff(y, x)\\n\\n# 특정 점에서의 기울기 계산\\npoint = 0  # 예를 들어 x=0에서의 기울기\\nslope_at_point = derivative.subs(x, point)\\nprint(slope_at_point)"}'

In [11]:
import json

def str_to_list(test_str):
    return json.loads(test_str)

print(str_to_list(test_str))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [1]:
from MindMap.LogManager import LogManager
from src.Systems.ActionCore import ActionCore

logManager = LogManager()
actionCore = ActionCore(logManager=logManager)

2025-01-23 05:33:59,265 - INFO - LogManager가 초기화 되었습니다..
2025-01-23 05:33:59,265 - INFO - Time: 2025-01-23_05-33-59
2025-01-23 05:33:59,266 - INFO - ==================================================

2025-01-23 05:33:59,268 - INFO - (ActionCore) Load: OK


In [6]:
import importlib
module_name = "Playground/compare"
module = importlib.import_module(module_name)


ModuleNotFoundError: No module named 'Playground/compare'

In [3]:
functions = {
            name: obj.__doc__.replace("  ", "") for name, obj in vars(module).items() if callable(obj) and not name.startswith("__")
        }

functions

{}

In [4]:
module

<module 'Playground' (namespace) from ['/workspaces/AI-Human/Playground']>

### Streaming 응답 예제

In [ ]:
from openai import OpenAI
from src.constants import OPENAI_APIKEY
client = OpenAI(api_key=OPENAI_APIKEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Hello, who are you and what do you do?"
        }
    ]
)

print(completion.choices[0].message.content)

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
completion = client.chat.completions.create(
      model="Mistral-Nemo-12b",
      messages=[
        {"role": "system", "content": "You are a helpful assistant. Always answer in Korean."},
        {"role": "user", "content": "OpenAI에 대해 알려줄래?"}
        ],
      stream=False
)

print(completion.choices[0].message.content)

In [ ]:
completion.choices[0].message.content

In [ ]:
import requests

url = "http://localhost:8080/v1/chat/completions"
payload = {
    "messages": [{"role": "user", "content": "Hello, how are you?"}],
    "stream": True
}

response = requests.post(url, json=payload, stream=True)

for line in response.iter_lines():
    if line.detail:
        print(line.decode('utf-8'))


In [ ]:
import uvicorn
import argparse
import torch
from threading import Thread
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [ ]:
MODEL_PATH="/Users/ckchoi/Desktop/chatbot/Mistral-Nemo-12b"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )

In [ ]:
messages=[
        {"role": "user", "content": "안녕?"}
    ]

In [ ]:
def generate_response(messages):
    inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    response = ''
    print(f"streamer: {streamer}")
    for text in streamer:
        response += text + ' '
        print(f"text: {text}")
        yield response

In [ ]:
generate_response(messages)

In [ ]:
for chunk in generate_response(messages):
    print(f"chunk: {chunk}")

In [ ]:
inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors="pt",
                        return_dict=True,
                        add_generation_prompt=True,
                        truncation=True,
                        max_length=1024).to('mps')
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
for text in streamer:
    print(type(text))